# Building the Master Dataset
This notebook will track the development of the master dataset upon which all our analyses will be based. Data from MITRE's CVE Project and IoT CVEs, the NVD, FIRST's EPSS data, and aggregated nation-state attack (NSA) data, as well as a smattering of IoT CVEs found from various articles (e.g. Check Point) will be merged via appropriate methods to produce a dataset that can offer a comprehensive analysis in the pursuit of building a holistic metric with which to help the (industrial) internet-of-things industry develop their interests in cybersecurity against advanced persistent threats (APT).

In the process of merging this data together, certain duplicate or empty values may be created that need to be dealt with accordingly. The actual analysis of the resulting dataset will be undertaken in a separate notebook (`master_analysis`) for clarity and in the interest of a separation of concerns.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
cves = pd.read_parquet(path='../data/CVE_Project/cvelistV5/cve_list_v3.parquet')
epss = pd.read_parquet(path='../data/EPSS/epss_data.parquet')
iots = pd.read_parquet(path='../data/MITRE/mitre_iot_cves_v1.parquet')
nvd = pd.read_parquet(path='../data/NVD/nvd_data_v1.parquet')
nsa = pd.read_parquet(path='../data/NSA/mock_nsa_data_v3.parquet')

In [13]:
cves.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250171 entries, 0 to 250170
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   cve_id       250171 non-null  string             
 1   description  250171 non-null  string             
 2   date_known   194415 non-null  datetime64[ns, UTC]
 3   cvss_v3      69256 non-null   float64            
 4   cvss_v3_cat  69256 non-null   category           
 5   vendor       250023 non-null  string             
 6   product      250057 non-null  string             
dtypes: category(1), datetime64[ns, UTC](1), float64(1), string(4)
memory usage: 11.7 MB


In [14]:
epss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260010 entries, 0 to 260009
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   cve_id      260010 non-null  string 
 1   epss        260010 non-null  float64
 2   percentile  260010 non-null  float64
dtypes: float64(2), string(1)
memory usage: 6.0 MB


In [15]:
iots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cve_id       1088 non-null   string
 1   description  1088 non-null   string
dtypes: string(2)
memory usage: 17.1 KB


In [2]:
nvd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   cve_id      1088 non-null   string             
 1   date_known  1088 non-null   datetime64[ns, UTC]
 2   cvss_v3     1088 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(1), string(1)
memory usage: 25.6 KB


In [17]:
nsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   attack_name            87 non-null     category      
 1   cve_list_date          67 non-null     datetime64[ns]
 2   date_of_first_exploit  67 non-null     datetime64[ns]
 3   patch_release_date     67 non-null     datetime64[ns]
 4   cvss                   67 non-null     float64       
 5   cvss_status            67 non-null     category      
 6   days_to_patch_release  67 non-null     Int64         
 7   days_to_first_exploit  67 non-null     Int64         
 8   year_start             80 non-null     Int64         
 9   year_end               80 non-null     Int64         
 10  attribution_group      48 non-null     category      
 11  attribution_state      79 non-null     category      
 12  cve_id                 83 non-null     string        
 13  descrip

In [18]:
cves.head()

,cve_id,description,date_known,cvss_v3,cvss_v3_cat,vendor,product
0,CVE-1999-0001,ip_input.c in BSD-derived TCP/IP implementatio...,2000-02-04 05:00:00+00:00,NaN,NaN,n/a,n/a
1,CVE-1999-0002,Buffer overflow in NFS mountd gives root acces...,1999-09-29 04:00:00+00:00,NaN,NaN,n/a,n/a
2,CVE-1999-0003,Execute commands as root via buffer overflow i...,1999-09-29 04:00:00+00:00,NaN,NaN,n/a,n/a
3,CVE-1999-0004,"MIME buffer overflow in email clients, e.g. So...",2000-02-04 05:00:00+00:00,NaN,NaN,n/a,n/a
4,CVE-1999-0005,Arbitrary command execution via IMAP buffer ov...,1999-09-29 04:00:00+00:00,NaN,NaN,n/a,n/a


In [19]:
epss.head()

,cve_id,epss,percentile
0,CVE-1999-0001,0.00383,0.73343
1,CVE-1999-0002,0.02080,0.89305
2,CVE-1999-0003,0.04409,0.92563
3,CVE-1999-0004,0.00917,0.83132
4,CVE-1999-0005,0.91963,0.99000


In [20]:
iots.head()

,cve_id,description
0,CVE-2024-29195,The azure-c-shared-utility is a C library for ...
1,CVE-2024-29055,Microsoft Defender for IoT Elevation of Privil...
2,CVE-2024-29054,Microsoft Defender for IoT Elevation of Privil...
3,CVE-2024-29053,Microsoft Defender for IoT Remote Code Executi...
4,CVE-2024-21324,Microsoft Defender for IoT Elevation of Privil...


In [11]:
nsa.head()

,cve_id,cve_list_date,date_of_first_exploit,patch_release_date,cvss,cvss_status,days_to_patch_release,days_to_first_exploit,attack_name,year_start,year_end,attribution_group,attribution_state,description
0,CVE-2014-0160,NaT,NaT,NaT,NaN,NaN,<NA>,<NA>,[Heartbleed Exploits],2014,2014,[None],[China],The (1) TLS and (2) DTLS implementations in Op...
1,CVE-2014-0630,2014-01-31 00:00:00+00:00,2015-12-23 00:00:00+00:00,2014-07-01 00:00:00+00:00,5.0,medium,151,691,"[BlackEnergy Attack on Ukraine, Stuxnet]",2015,2015,[Sandworm],"[Russia, US]",EMC Documentum TaskSpace (TSP) 6.7SP1 before P...
2,CVE-2014-4166,2014-10-07 00:00:00+00:00,2015-12-23 00:00:00+00:00,2015-01-12 00:00:00+00:00,5.0,medium,97,442,"[BlackEnergy Attack on Ukraine, Stuxnet]",2015,2015,[Sandworm],"[Russia, US]",Cross-site scripting (XSS) vulnerability in th...
3,CVE-2014-6485,2014-10-14 00:00:00+00:00,2015-12-23 00:00:00+00:00,2014-12-10 00:00:00+00:00,7.5,high,57,435,"[BlackEnergy Attack on Ukraine, Stuxnet]",2015,2015,[Sandworm],"[Russia, US]",Unspecified vulnerability in Oracle Java SE 8u...
4,CVE-2014-8361,2014-09-23 00:00:00+00:00,2014-10-01 00:00:00+00:00,2015-05-05 00:00:00+00:00,9.3,critical,224,8,[Mirai Botnet],<NA>,<NA>,[],[],The miniigd SOAP service in Realtek SDK allows...


## Merge Strategy
The goal of this merge strategy is to combine data from five distinct dataframes: `iots`, `cves`, `epss`, `nvd`, and `nsa`. The merge will begin by a series of leftward merges into the `iots` dataframe—since we're focused on IoT CVEs—to preserve its integreity as the core base of our master data. We will finalize the working copy of the master data through an outward merge with `nsa` since this contains relevant NSA data that we don't want to filter out. Sometimes, if the merge key is not the only column of the same name across both tables, and if the merge key is the only column we're merging from, then both columns from both tables will be merged and each will be appended with an `_x` and a `_y`, respectively. Because of this, it may be necessary to combine these columns and drop the duplicates after each successful merge.

In [2]:
# Helper function to combine columns and drop their duplicates
def combine_and_drop(df, cols: dict):
    """
    This function takes a dataframe and a dictionary containing a list of
    columns to merge and drop whose key is the name of the resultant column.
    """
    # Loop through dictionary to combine columns
    for result, source in cols.items():
        if "date_known" in result:
            # Take earliest date between the two
            df[result] = df[[source[0], source[1]]].min(axis=1)
        else:
            df[result] = df[source[0]].combine_first(df[source[1]])
    # Drop duplicate columns
    df = df.drop(
        columns=[
            col for result, source in cols.items() for col in source if col != result
        ]
    )
    return df

### Merge with `cves`

In [3]:
df = iots.merge(cves, on='cve_id', how='left')

# Dictionary of columns to combine
cols_to_combine = {
    'description': ['description_x', 'description_y']
}

df = combine_and_drop(df, cols_to_combine)

### Merge with `epss`

In [4]:
df = df.merge(epss, on='cve_id', how='left')

### Merge with `nvd`

In [5]:
df = df.merge(nvd, on='cve_id', how='left')

cols_to_combine = {
    'date_known': ['date_known_x', 'date_known_y'],
    'cvss_v3': ['cvss_v3_x', 'cvss_v3_y']
}

df = combine_and_drop(df, cols_to_combine)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   cve_id                 1088 non-null   string             
 1   cvss_v3_cat            481 non-null    category           
 2   vendor                 1088 non-null   string             
 3   product                1088 non-null   string             
 4   epss                   1088 non-null   float64            
 5   percentile             1088 non-null   float64            
 6   date_known             1088 non-null   datetime64[ns, UTC]
 7   cvss_v3                1088 non-null   float64            
 8   date_of_first_exploit  0 non-null      datetime64[ns, UTC]
 9   patch_release_date     0 non-null      datetime64[ns, UTC]
 10  days_to_patch_release  0 non-null      Int64              
 11  days_to_first_exploit  0 non-null      Int64            

### Merge with `nsa`

In [7]:
df = df.merge(nsa, on='cve_id', how='outer')

cols_to_combine = {
    'description': ['description_x', 'description_y'],
    'cvss_v3': ['cvss_v3', 'cvss'],
    'cvss_v3_cat': ['cvss_v3_cat', 'cvss_status'],
    'date_known': ['date_known', 'cve_list_date']
}

df = combine_and_drop(df, cols_to_combine)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1136 entries, 0 to 1135
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   cve_id                 1136 non-null   string             
 1   date_of_first_exploit  30 non-null     datetime64[ns, UTC]
 2   patch_release_date     30 non-null     datetime64[ns, UTC]
 3   days_to_patch_release  30 non-null     Int64              
 4   days_to_first_exploit  30 non-null     Int64              
 5   attack_name            48 non-null     object             
 6   year_start             37 non-null     Int64              
 7   year_end               37 non-null     Int64              
 8   attribution_group      48 non-null     object             
 9   attribution_state      48 non-null     object             
 10  cvss_v3_cat            515 non-null    category           
 11  vendor                 1135 non-null   string           

## Re-Merge Procedure
Due to the way the `nsa` merge expands the observation count, we need to re-merge `cves` and `epss` into the dataset to capture a few CVSS and EPSS scores, dates, vendors and products (if available). If `cves` doesn't have this information, we'll update the API caller in `nvd_extraction` and pass in the new list of CVEs we have in this master dataset, then merge an updated version of `nvd_data` back into our master dataset.

In [10]:
# Remerge CVEs
df = df.merge(cves, on='cve_id', how='left')

cols_to_combine = {
    'cvss_v3_cat': ['cvss_v3_cat_x', 'cvss_v3_cat_y'],
    'vendor': ['vendor_x', 'vendor_y'],
    'product': ['product_x', 'product_y'],
    'date_known': ['date_known_x', 'date_known_y'],
    'cvss_v3': ['cvss_v3_x', 'cvss_v3_y'],
    'description': ['description_x', 'description_y']
}

df = combine_and_drop(df, cols_to_combine)

In [12]:
# Remerge EPSS
df = df.merge(epss, on='cve_id', how='left')

cols_to_combine = {
    'epss': ['epss_x', 'epss_y'],
    'percentile': ['percentile_x', 'percentile_y']
}

df = combine_and_drop(df, cols_to_combine)

This is that same CVE ID (`CVE-2022-26658`) that doesn't exist in MITRE's CVE Project, the NVD, VulnDB, or CVEFeed.io. MITRE's website says that this ID is `reserved`, meaning that it hasn't been mapped to an actual vulnerability yet. Curiously, it still a list date, patch release data, first exploitation date, CVSS score, and an association with Volt Typhoon. Obviously, this is something we'll have to look further into. This is likely the only CVE ID that is reserved, considering that only reserved IDs are missing descritions and that all other IDs in our dataset have respective descriptions.

In [15]:
# Get rid of the non-existant vulnerability
df = df.drop(df[df['cve_id'] == 'CVE-2022-26658'].index)

## Adding CVEs from Checkpoint Article
Based on missing CVSS scores for $11$ CVE IDs, we'll take those $11$ observations, save them into their own small dataset, import it into `nvd_extraction`, and call NVD's API to grab what we need. At the same time, we'll add the $2$ CVEs found in the Check Pount article ([reread it here](https://blog.checkpoint.com/security/the-tipping-point-exploring-the-surge-in-iot-cyberattacks-plaguing-the-education-sector/)) that weren't in our data already because MITRE doesn't have their CVSS scores.

In [16]:
count = len(df[(df['cve_id'].notnull()) & (df['cvss_v3'].isnull())])
print(f'We need to gather CVSS scores for {count} CVEs from the NVD.')

We need to gather CVSS scores for 14 CVEs from the NVD.


In [17]:
# Create Check Point CVE dataset
cp = {
    'cve_id': [
        'CVE-2015-2051',
        'CVE-2016-6277',
        'CVE-2022-37061'
    ]
}
article_cves = pd.DataFrame(cp)

# Pull out mini dataset
missing_cves = df[(df['cve_id'].notnull()) & (df['cvss_v3'].isnull())]
missing_cves = missing_cves['cve_id']
missing_cves = missing_cves.to_frame()
missing_cves = pd.concat([missing_cves, article_cves], ignore_index=True)

# Save the mini dataset
missing_cves.to_parquet(path='../data/miniset_cves_v2.parquet')

In [15]:
# Loading in the mini response taken from NVD
mini = pd.read_parquet(path='../data/NVD/mini_nvd_response_v2.parquet')

In [16]:
# Remerge NVD
df = df.merge(mini, on='cve_id', how='left')

cols_to_combine = {
    'date_known': ['date_known_x', 'date_known_y'],
    'cvss_v3': ['cvss_v3_x', 'cvss_v3_y']
}

df = combine_and_drop(df, cols_to_combine)

## Standardizing Null Values

In [17]:
cols_of_int = ['vendor', 'product']
df[cols_of_int] = df[cols_of_int].replace('n/a', pd.NA)

## Validating CVSS Scores

In [25]:
def map_cvss_to_category(score):
    if score >= 9.0:
        return 'critical'
    elif score >= 7.0:
        return 'high'
    elif score >= 4.0:
        return 'medium'
    elif score > 0.0:
        return 'low'
    elif score == 0.0:
        return 'none'
    return None

df['cvss_v3_cat'] = df['cvss_v3'].apply(map_cvss_to_category)

cvss_cats = ['none', 'low', 'medium', 'high', 'critical']
df['cvss_v3_cat'] = pd.Categorical(
    df['cvss_v3_cat'],
    categories=cvss_cats,
    ordered=True
)

## Reordering Columns for Readability

In [26]:
new_order = [
    'cve_id',
    'description',
    'epss',
    'percentile',
    'cvss_v3',
    'cvss_v3_cat',
    'date_known',
    'patch_release_date',
    'date_of_first_exploit',
    'days_to_patch_release',
    'days_to_first_exploit',
    'vendor',
    'product',
    'attack_name',
    'year_start',
    'year_end',
    'attribution_group',
    'attribution_state'
]
df = df[new_order]

## Sorting the Data
The following section floats meaningful data to the top and sinks empty values to the bottom. It does this by sorting a dummy column that counts the number of non-null values across each row. After this column has been sorted, the dataset is sorted according to `cve_id`.

In [27]:
# Create column of non-null counts
df['nn_count'] = df.notnull().sum(axis=1)

# Sort table
df = df.sort_values(by=['nn_count', 'cve_id'], ascending=[False, True])

# Drop dummy column
df = df.drop(columns=['nn_count'])
df.head(3)

,cve_id,description,epss,percentile,cvss_v3,cvss_v3_cat,date_known,patch_release_date,date_of_first_exploit,days_to_patch_release,days_to_first_exploit,vendor,product,attack_name,year_start,year_end,attribution_group,attribution_state
0,CVE-2019-0729,An Elevation of Privilege vulnerability exists...,0.00267,0.68003,9.8,critical,2019-03-05 00:00:00+00:00,NaT,NaT,<NA>,<NA>,Microsoft,Java SDK for Azure IoT,NaN,<NA>,<NA>,NaN,NaN
1,CVE-2019-0741,An information disclosure vulnerability exists...,0.00374,0.73050,7.5,high,2019-03-05 00:00:00+00:00,NaT,NaT,<NA>,<NA>,Microsoft,Java SDK for Azure IoT,NaN,<NA>,<NA>,NaN,NaN
2,CVE-2019-1010009,DGLogik Inc DGLux Server All Versions is affec...,0.01260,0.85854,9.8,critical,2019-07-15 02:00:46+00:00,NaT,NaT,<NA>,<NA>,DGLogik Inc,DGLux Server,NaN,<NA>,<NA>,NaN,NaN


## Dropping Duplicate Observations

In [29]:
dups = df.duplicated().sum()
print(f'There are {dups} duplicate observations in the dataset.')

# Drop duplicates
df = df.drop_duplicates(keep='first')
dups = df.duplicated().sum()
print(f'After the drop, there are now {dups} duplicate observations in the dataset.')

There are 0 duplicate observations in the dataset.
After the drop, there are now 0 duplicate observations in the dataset.


The following table shows the four observations that do not include CVE IDs and their associated information. If we find them, adding this information will be relatively straightforward.

In [28]:
df[df['cve_id'].isnull()]

,cve_id,description,epss,percentile,cvss_v3,cvss_v3_cat,date_known,patch_release_date,date_of_first_exploit,days_to_patch_release,days_to_first_exploit,vendor,product,attack_name,year_start,year_end,attribution_group,attribution_state


## Resetting Index

In [23]:
df = df.reset_index(drop=True)

In [30]:
df.head(2)

,cve_id,description,epss,percentile,cvss_v3,cvss_v3_cat,date_known,patch_release_date,date_of_first_exploit,days_to_patch_release,days_to_first_exploit,vendor,product,attack_name,year_start,year_end,attribution_group,attribution_state
0,CVE-2019-0729,An Elevation of Privilege vulnerability exists...,0.00267,0.68003,9.8,critical,2019-03-05 00:00:00+00:00,NaT,NaT,<NA>,<NA>,Microsoft,Java SDK for Azure IoT,NaN,<NA>,<NA>,NaN,NaN
1,CVE-2019-0741,An information disclosure vulnerability exists...,0.00374,0.73050,7.5,high,2019-03-05 00:00:00+00:00,NaT,NaT,<NA>,<NA>,Microsoft,Java SDK for Azure IoT,NaN,<NA>,<NA>,NaN,NaN


In [31]:
df.tail(2)

,cve_id,description,epss,percentile,cvss_v3,cvss_v3_cat,date_known,patch_release_date,date_of_first_exploit,days_to_patch_release,days_to_first_exploit,vendor,product,attack_name,year_start,year_end,attribution_group,attribution_state
1086,CVE-2023-34367,Windows 7 is vulnerable to a full blind TCP/IP...,0.00369,0.72886,6.5,medium,2023-06-14 00:00:00+00:00,NaT,NaT,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,NaN,NaN
1087,CVE-2023-47882,The Kami Vision YI IoT com.yunyi.smartcamera a...,0.00069,0.31145,7.1,high,2023-12-27 00:00:00+00:00,NaT,NaT,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,NaN,NaN


## Recalcuting Days to Patch and First Exploit
This is just to check that all the merges went smoothly.

In [32]:
df['days_to_patch_release'] = (df['patch_release_date'] - df['date_known']).dt.days
df['days_to_first_exploit'] = (df['date_of_first_exploit'] - df['date_known']).dt.days


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   cve_id                 1088 non-null   string             
 1   description            1088 non-null   string             
 2   epss                   1088 non-null   float64            
 3   percentile             1088 non-null   float64            
 4   cvss_v3                1088 non-null   float64            
 5   cvss_v3_cat            1088 non-null   category           
 6   date_known             1088 non-null   datetime64[ns, UTC]
 7   patch_release_date     0 non-null      datetime64[ns, UTC]
 8   date_of_first_exploit  0 non-null      datetime64[ns, UTC]
 9   days_to_patch_release  0 non-null      float64            
 10  days_to_first_exploit  0 non-null      float64            
 11  vendor                 1024 non-null   string           

## Saving Master Data
That's it! This is the state of our data so far.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   cve_id                 1151 non-null   string             
 1   description            1151 non-null   string             
 2   epss                   1151 non-null   float64            
 3   percentile             1151 non-null   float64            
 4   cvss_v3                1151 non-null   float64            
 5   cvss_v3_cat            1151 non-null   category           
 6   date_known             1151 non-null   datetime64[ns, UTC]
 7   patch_release_date     47 non-null     datetime64[ns, UTC]
 8   date_of_first_exploit  47 non-null     datetime64[ns, UTC]
 9   days_to_patch_release  47 non-null     Int64              
 10  days_to_first_exploit  47 non-null     Int64              
 11  vendor                 1044 non-null   string           

In [23]:
df.to_parquet(path='../data/master_data_v1.parquet', index=None)
df.to_csv('../data/master_data_v1_indexed.csv', index=True)

In [10]:
df[df['cve_id'].duplicated()]

,cve_id,cvss_v3_cat,vendor,product,epss,percentile,date_known,cvss_v3,attack_name,date_of_first_exploit,patch_release_date,days_to_patch_release,days_to_first_exploit,year_start,year_end,attribution_group,attribution_state,description
